In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [3]:
class CaseModel(nn.Module):
    def __init__(self, embedding_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(CaseModel, self).__init__()    
        self.embedding_dim = embedding_dim
        self.hid1= embedding_dim*2
        self.hid2 = embedding_dim//2
        self.fc1 = nn.Linear(self.embedding_dim, self.hid1)
        self.fc2 = nn.Linear(self.hid1, self.hid2)
        self.hidden2tag = nn.Linear(self.hid2, tagset_size)
    
    def forward(self, x):
        y = F.relu(self.fc1(x))
        y = F.relu(self.fc2(y))
        y = self.hidden2tag( y )
        y = F.log_softmax(y, dim=1)
        return y

In [4]:
border=0.8

In [5]:
dict_path = "../models/utterance/"
dict_name = "group_border={0}_2.pickle".format(border)
dictM = DataManager(dict_path)
group_dict = dictM.load_data(dict_name)

success load : ../models/utterance/group_border=0.8_2.pickle


In [6]:
def search_word(word):
    word = clean_text(word)
    found_list = []
    for group_key in group_dict.keys():
        if word in group_dict[group_key]:
            found_list.append(group_key)
    return found_list

In [7]:
from gensim.models import KeyedVectors
w2v_path = "../../corpus/w2v/"
# fasttext
# https://qiita.com/Hironsan/items/513b9f93752ecee9e670
# w2v_name =  "dep-ja-300dim"
w2v_name =  "model.vec"
w2v_model = KeyedVectors.load_word2vec_format(w2v_path+w2v_name)

[2152] 2022-01-06 00:48:23,715 Info gensim.models.keyedvectors :loading projection weights from ../../corpus/w2v/model.vec
[2152] 2022-01-06 00:49:26,580 Info gensim.utils :KeyedVectors lifecycle event {'msg': 'loaded (351122, 300) matrix of type float32 from ../../corpus/w2v/model.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-01-06T00:49:26.579736', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-Ubuntu-18.04-bionic', 'event': 'load_word2vec_format'}


In [8]:
group2index = dict(zip( group_dict.keys(), range(len(group_dict.keys())) ))
index2group = dict(zip(range(len(group_dict.keys())), group_dict.keys() ))

In [9]:
model_path = "../models/utterance/"
model_name = "case_frame_{0}_2.pickle".format(border)
modelM = DataManager(model_path)
cmodel = modelM.load_data(model_name)

success load : ../models/utterance/case_frame_0.8_2.pickle


In [10]:
def classify_word(word):
    word = clean_text(word)
    if word not in w2v_model:
        return ""
    with torch.no_grad():
        vector = torch.tensor([w2v_model[word]]).cuda()
        pred = np.array(cmodel(vector).cpu()).argmax()
    
    return index2group[pred]

In [32]:
word = "オリンピック"

In [33]:
search_word(word)

[]

In [34]:
classify_word(word)

'<イベント>'

In [14]:
group_filename = "../../corpus/case_frame/easy_case_frame.csv"

In [15]:
def register_triple(case_frane:dict, V, C, Noun):
    # 動詞の確認
    if V not in case_frame:
        case_frame[V] = dict()

    # 格の確認
    if C not in case_frame[V]:
        case_frame[V][C] = set()
    
    # 名詞の登録
    case_frame[V][C].add(Noun)

In [16]:
import csv
case_frame = dict()
unique_nouns = dict()

with open(group_filename, mode="r") as f:
    reader = csv.reader(f)
    for row in reader:
        V = row[0]
        C = row[1]
        for i in range(2, len(row), 2):
            if "Unnamed" in row[i] or "" == row[i]:
                continue
            Noun = row[i]
            register_triple(case_frame, V, C, row[i])

            # 有効 などの登録
            if Noun not in group_dict.keys():
                if Noun not in unique_nouns:
                    unique_nouns[Noun] = set()
                unique_nouns[Noun].add(V)

In [17]:
case_frame_name = "case_frame.pickle"
unique_name = "unique_nouns.pickle"
group_name = "group.pickle"
data_path = "../X_y_data/utterance/"

In [18]:
dataM = DataManager(data_path)
dataM.save_data(unique_name, unique_nouns)
dataM.save_data(case_frame_name, case_frame)
# dataM.save_data(group_name, group_dict)

success save : ../X_y_data/utterance/unique_nouns.pickle
success save : ../X_y_data/utterance/case_frame.pickle


In [73]:
def is_declinable(token):
    pos = token.tag_.split("-")
    try:
        if pos[0] == "動詞" or pos[0] == "形容詞" or pos[2] == "サ変可能":
            # print(token.lemma_)
            return True
        else:
            False
    except:
        return False


# case_set = set( "が　を　に　で は".split() )

def extract_RDF_triple(text):
    text = clean_text(text)
    doc = nlp(text)
    triple_list = []
    for i, token in enumerate( doc ):
        # if token.pos_ in ["VERB", "ADJ"]:
        if token.pos_=="VERB":
        # if is_declinable(token):
            phrase = ""
            for c in token.children:
                if c.dep_ in ["nsubj", "obj"]:
                    noun = c.lemma_
                    for c2 in c.children:
                        # if c2.dep_ == "case" and c2.orth_ in case_set:
                        if c2.dep_ == "case":
                            case = c2.orth_
                            if case == "は":
                                case = "が"
                            triple_list.append( (token.lemma_, case, noun) )
    return triple_list



In [76]:
t = "インターネットは排出量が多い"
t = "固有抽出表現は、文から日付や地名等の固有表現を抽出する処理です。"
# t = "マドリッドの原宿、表参道エリアを中心に人気専門店が続々登場するなど、今年一躍話題になったポップコーンですが、日本穀物検定協会が金曜ロードSHOW!とコラボレーションしたきょうは会社休みます。を12月に発売するそうです。"
triples = extract_RDF_triple(t)
triples

[('抽出', 'を', '表現')]

In [54]:
def search_triple(V, C, N):
    if N in case_frame[V][C]:
        return True
    else:
        return False

In [55]:
# True : 用法は問題ない
# False : 問題あり
def judge_triple(triple):
    V = triple[0]
    C = triple[1]
    noun = triple[2]

    # Vが登録されていないならパスしておくかな(一旦)
    if V not in case_frame:
        # print("not registered V :", V)
        return True
    
    # C が登録されていないなら，アウト
    if C not in case_frame[V]:
        # print("not registered V, C :", V, C)
        return False
    
    # 名詞はユニークで，且つ動詞とペアになるか？
    if noun in unique_nouns :
        # print("unique noun", noun)
        if noun in case_frame[V][C]:
            # 存在するなら良かったね
            if search_triple(V, C, noun):
                return True
    
    # 名詞がどこかのグループに属する
    group_ = search_word(noun)
    if len(group_) > 0:
        # 1つでも引っかかればOK
        for group in group_:
            if search_triple(V, C, group):
                return True
        return False
    # グループを推定
    else:
        group = classify_word(noun) 
        return search_triple(V, C, group)

In [56]:
for triple in triples:
    print(triple)
    print(judge_triple(triple))
    print("----")

In [57]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [58]:
error = "Semantic error"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [59]:
y_pred = []
for utt in tqdm(sys_utt):
    is_valid = True

    triples = extract_RDF_triple(utt)
    for triple in triples:
        if not judge_triple(triple):
            is_valid = False
            break
    if is_valid:
        y_pred.append(0)
    else:
        y_pred.append(1)

100%|██████████| 1386/1386 [00:22<00:00, 62.18it/s]


In [63]:
score(y, y_pred)

confusion matrix = 
 [[847 531]
 [  6   2]]
accuracy =  0.6125541125541125
precision =  0.00375234521575985
recall =  0.25
f1 score =  0.0073937153419593345


- 複文への分類がキツイ可能性を考えた
- 複文への分類が生きている場合


        confusion matrix = 
        [[888 490]
        [  6   2]]
        accuracy =  0.6421356421356421
        precision =  0.0040650406504065045
        recall =  0.25
        f1 score =  0.008

- 複文への分類はあきらめた

        confusion matrix = 
        [[921 457]
        [  6   2]]
        accuracy =  0.665945165945166
        precision =  0.004357298474945534
        recall =  0.25
        f1 score =  0.008565310492505354

In [80]:
w = "人たち"
print( search_word(w) ) 
print(classify_word(w))

['<ヒト>']
<ヒト>


In [64]:
for t, p, utt in zip(y, y_pred, sys_utt):
    if t==0 and p==1:
        print(utt)
        print(extract_RDF_triple(utt))
        print("--------")

お、そうですか。　えっと、四国は行かれたことありますか？
[('ある', 'が', '四国')]
--------
お兄さんに生まれると、頼りがいのある性格になるのかな。
[('生まれる', 'に', 'お兄さん'), ('なる', 'に', '性格')]
--------
あ　そうでしたか。　ところで、九州に行かれたことありますか？
[('行く', 'に', '九州'), ('ある', 'で', '所')]
--------
熱中症に気をつけか？？
[('つける', 'に', '熱中症'), ('つける', 'を', '気')]
--------
熱中症に気をつけたいのです
[('つける', 'に', '熱中症'), ('つける', 'を', '気')]
--------
朝からスイミングで泳ぎます？？
[('泳ぐ', 'で', 'スイミング')]
--------
スイミングを続けるかもしれない
[('続ける', 'を', 'スイミング')]
--------
何か普段の生活で気をつけていることはありますか。
[('つける', 'で', '生活'), ('つける', 'を', '気'), ('ある', 'が', 'こと')]
--------
和食、洋食、中華どれが食べたいですか？
[('食べる', 'が', 'どれ')]
--------
退屈させてしまいましたか。うまくコミュニケーション取れるように頑張るので、お付き合いください。話題を変えましょう、最近観た映画は何ですか？
[('変える', 'を', '話題')]
--------
昨日は何を食べましたか？
[('食べる', 'が', '昨日'), ('食べる', 'を', '何')]
--------
なんと。　ところで、お花見って行ったことありますか？
[('ある', 'で', '所')]
--------
日本では身体を使ったものを指すけど、本来思考力や計算力もスポーツに含まれるよ。
[('使う', 'を', '身体'), ('指す', 'で', '日本'), ('指す', 'が', '日本'), ('指す', 'を', '物'), ('含む', 'に', 'スポーツ')]
--------
日本の国花は桜だけど、法律で決められたものじゃないんだよ。
[('決める', 'で', '法律')]
----

In [88]:
t = "人たち"
from spacy import displacy
displacy.render(nlp(t), style='dep', jupyter=True, options={'compact':True, 'distance': 90})

In [83]:
def compound_noun(token):
    base = token.lemma_
    for child in token.children:
        if child.dep_ == "compound":
            return child.lemma_ + base